In [ ]:
volumes:
  pgdata:

  postgres:
    container_name: postgres
    image: postgres
    volumes:
    - pgdata:/var/lib/postgresql/data
    environment:
      POSTGRES_PASSWORD: uworx

# Reflection

- inspect and manipulate metadata of types and members (such as fields, properties, methods, constructors, etc.)
- Java had reflection (API) but annotations were added later; C# always had reflection and attributes (annotation in C# / .NET is called attributes)
- Three building blocks
    - Query Information; type, properties, methods, their modifiers etc; Attributes compliment Reflection nicely
    - Access Properties; values of given object and invoke methods
    - the ability to dynamically create instances of types

In [ ]:
using System.ComponentModel.DataAnnotations;
using System.Reflection;

class Person
{
    [Required]
    public int PersonId { get; set; }
    public string FirstName { get; set; }
    public string LastName { get; set; }
}

var person = new Person() { FirstName = "Khurram", LastName = "Aziz" };
Type type = person.GetType();       //we have typeof() operator; we can do type == typeof(Person) for type checking etc
Console.WriteLine(type);

foreach(var p in type.GetProperties())  // why its type.GetProperties() and not type.Properties ? 
{
    Console.Write($"{p.Name} : {p.PropertyType}");
    bool isRequired = Attribute.IsDefined(p, typeof(RequiredAttribute));

    if (isRequired)
        Console.WriteLine($" Required; Value: {p.GetValue(person)}");
    else
        Console.WriteLine();
}

In [ ]:
// LINQ and Reflection

using System.Reflection;

var assembly = Assembly.Load("System.Private.CoreLib");//, Version=7.0.0.0, Culture=neutral, PublicKeyToken=7cec85d7bea7798e");
                                                        // assembly full name
var query = from type in assembly.GetTypes()
            where type.IsPublic
            from method in type.GetMethods()
            where method.ReturnType.IsArray == true
                || (
                    method.ReturnType.GetInterface(typeof(System.Collections.Generic.IEnumerable<>).FullName!) != null
                    && method.ReturnType.FullName != "System.String"
                )
            group method.ToString() by type.ToString();

foreach (var groupOfMethods in query.Take(10))  // lets just take top 10
{
    Console.WriteLine("Type: {0}", groupOfMethods.Key);
    foreach (var method in groupOfMethods)
        Console.WriteLine("  {0}", method);
}

- Replacing / Complimenting Dependency Injection
    - https://autofac.org
    - https://github.com/khellang/Scrutor

In [ ]:
//autofac example

var builder = new ContainerBuilder();
// Register individual components
builder.RegisterInstance(new TaskRepository())
       .As<ITaskRepository>();
builder.RegisterType<TaskController>();
builder.Register(c => new LogManager(DateTime.Now))
       .As<ILogger>();
 
// Scan an assembly for components
builder.RegisterAssemblyTypes(myAssembly)
       .Where(t => t.Name.EndsWith("Repository"))
       .AsImplementedInterfaces();
 
var container = builder.Build();

How to create instances, invoke methods; lets discuss this in next session
__Further Readings__
- https://learn.microsoft.com/en-us/dotnet/csharp/advanced-topics/reflection-and-attributes

# Nuget of the day

- https://www.nuget.org/packages/Scrutor
- https://github.com/khellang/Scrutor

In [ ]:
services.Scan(scan =>
    scan.FromCallingAssembly()
        .AddClasses()
        .AsMatchingInterface()
        .WithTransientLifetime());  // optional

# Microservices

- https://learn.microsoft.com/en-us/dotnet/architecture/microservices
- Microservices are small, independent, and loosely coupled.
- Each service is a separate codebase,  __A single small team of developers can write and maintain a service__
- Services can be deployed independently. __A team can update an existing service without rebuilding and redeploying the entire application__
- __Supports polyglot programming__. For example, services don't need to share the same technology stack, libraries, or frameworks
- Services communicate with each other by using well-defined APIs. Internal implementation details of each service are hidden from other services
- __*Services are responsible for persisting their own data or external state*__. This differs from the traditional model, where a separate data layer handles data persistence
- __Cloud__: https://learn.microsoft.com/en-us/azure/architecture/microservices

In [ ]:
/*
How services will communicate?

Challenges of Distributed Computing

    Complexity
    Service Discovery; DNS
    Network Latencies
    Bottlenecks
    Data Management
    Security
    Monitoring, Observability
    Resource Management
    Testing

    Dividing work
    Orchestration
*/

//CNCF

# The Dark Side of the Moon

In [ ]:
// Does it matter in which language some microservice is written in ?

<img src=images/dark-side.png>

# Extensions

<img src=images/extensions.png>

- https://learn.microsoft.com/en-us/dotnet/core/diagnostics/observability-with-otel
- https://devblogs.microsoft.com/dotnet/building-resilient-cloud-services-with-dotnet-8/
- https://github.com/dotnet/extensions

# Nuget: Dapplo.Jira

In [ ]:
#r "nuget: Dapplo.Jira, *-*"

In [ ]:
var url = "https://YOUR-COMPANY.atlassian.net";
var user = "you@YOUR-COMPNAY.com";
var token = "YOUR-TOKEN";

In [ ]:
using Dapplo.Jira;
using Dapplo.Jira.Query;

IJiraClient jiraClient = JiraClient.Create(new Uri(url));
jiraClient.SetBasicAuthentication(user, token);

var jql = Where.Text.Contains("BuilderService");
var searchResult = await jiraClient.Issue.SearchAsync(jql);

foreach (var issue in searchResult.Issues)
{
    Console.WriteLine($"{issue.Key} [{issue.Fields.Status.Name}] {issue.Fields.Summary}");

    var workLogs = await jiraClient.WorkLog.GetAsync(issue.Key);
    
    foreach (var workLog in workLogs)
        Console.WriteLine($"\t{workLog.Id} {workLog.Author.EmailAddress} [{workLog.Author.AccountId}] {workLog.Started} {workLog.TimeSpent}");
}

In [ ]:
searchResult.Issues.FirstOrDefault()

In [ ]:
var issue = await jiraClient.Issue.GetAsync("PROJ-1"); // some jira ticket id
issue

In [ ]:
foreach(var f in issue.Fields.CustomFields)
    Console.WriteLine(f);

In [ ]:
var fields = await jiraClient.Server.GetFieldsAsync();
foreach(var f in fields.OrderBy(s => s.Name))
    Console.WriteLine(f.Name);

In [ ]:
var storyPointField = fields.FirstOrDefault(x => x.Name == "Story Points");
var planningCodeField = fields.FirstOrDefault(x => x.Name == "Planning Code");

planningCodeField

In [ ]:
KeyValuePair<string, object> storyPoints = issue.Fields.CustomFields.FirstOrDefault(x => x.Key == storyPointField.Id);
KeyValuePair<string, object> planningCode = issue.Fields.CustomFields.FirstOrDefault(x => x.Key == planningCodeField.Id);
//lets inspect these variables in Notebook

In [ ]:
//storyPoints value is a number that we are looking for
Console.WriteLine(storyPoints.Value);

//planningCode value is a shape; we need value field out of that shape; a string
//we dont need to create a type for this as C#/.NET has dynamic/DLR thing
dynamic planningCodeValue = planningCode.Value;
Console.WriteLine(planningCodeValue.value);

# JiraWorkLogs

## Github
- https://github.com/khurram-uworx/jiraworklogs
- Issues / Closed Issues
- Pull Requests / Closed PRs
- Opened Issues
- Discussions, Wiki
- Actions & Security

## Requirements and Features

__Why (Context)__
- Business Case / Actors / Concept / Need
- Empowering Managers
- Empowering Team
- Transparency

__What (Requirement Specifications)__
- Jira Time Report

__How (Functional Spectifications / Design)__
- Relational Database / Querying + Reporting Capability
- Long running app / "service" that retreives Jira Worklogs
- Web; jQuery + Bootstrap; we can use ChatGPT and Internet for ready made ideas and html / css code
- Deployment Units / Components
    - PostgreSQL for RDBMS
    - MVC for Web App; we dont need fancy Angular/React/Vue/Blazor app (for time being)
    - Redis for Cache
    - RabbitMQ for messaging
- *Mitigating Risks*; Open Source Culture

## Observability

- Tracing, Telemetry, Metrics
- ILogger, System.Diagnostics (Activity) and System.Diagnostics.Metrics (New, Meter)
- https://github.com/open-telemetry/opentelemetry-dotnet/tree/main/examples
    - https://github.com/open-telemetry/opentelemetry-dotnet/blob/main/examples/MicroserviceExample/WebApi/Startup.cs
    - https://github.com/open-telemetry/opentelemetry-dotnet/blob/main/src/OpenTelemetry.Instrumentation.AspNetCore/README.md
    - https://github.com/open-telemetry/opentelemetry-dotnet/blob/main/src/OpenTelemetry.Exporter.Prometheus.AspNetCore/README.md
    - https://github.com/open-telemetry/opentelemetry-dotnet/blob/main/src/OpenTelemetry.Exporter.Zipkin/README.md

In [ ]:
//Fluent APIs
class Team {}
class Employee
{
    public Team AssignTeam(Team team) => null;
}

class HumanResource
{
    public Employee AddEmployee(Employee employee) => null;
}

var e = new Employee();
var t = new Team();
var hr = new HumanResource();
hr.AddEmployee(e).AssignTeam(t);     // or we can write second method in next line

In [ ]:
//Builder and configuration
abstract class Builder {}
class SpecializedBuilder : Builder {}

static void Configure(this Builder builder, Action<SpecializedBuilder> action) // extension method
{
    //do something with builder
    action(builder as SpecializedBuilder);
    //do something more with builder
}

Builder builder = new SpecializedBuilder();
//add some specialized stuff in builder instance
builder.Configure(b =>
{
    //b is builder; all the added stuff is available here
});     //inversion of control

// we can have more Configure methods; say how to setup metrices, telemetry and so on if our builder is open telemetry

https://learn.microsoft.com/en-us/dotnet/api/microsoft.extensions.configuration.configurationbuilder

In [ ]:
class ExpensiveResource : IDisposable
{
    public void Dispose() {}
}

using(var resource = new ExpensiveResource())
{

} //block of code

// we can using() block to basically wrap / surround things in block; as pattern
// when we wrap things in some block; try/catch or using; we see lot of changes in code when committing
// newer simplified sytnax

using var resource2 = new ExpensiveResource(); // it will be available for the rest of the block

In [ ]:
// NULL is your friend

class Something { public void SomeMethod() {} }
Something ComingFromSomewhere() => null;

var something = ComingFromSomewhere(); //something can be null

if (null == something) something = new Something(); // if it was null; here's the alternative value
var something1 = ComingFromSomewhere() ?? new Something(); // if its null what's the alternative value

if (null != something) something.SomeMethod();
something?.SomeMethod();        // only call when something is not null

## Cache

- https://github.com/Azure-Samples/azure-cache-redis-samples
    - https://github.com/Azure-Samples/azure-cache-redis-samples/blob/main/quickstart/aspnet-core/ContosoTeamStats/RedisConnection.cs
- https://getbootstrap.com/docs/4.0/components/pagination

In [ ]:
// Why Resilience is important?

// Generics / Action<T> and Func<T>; wrapping them in something cool
    // Redis retry code

In [ ]:
//Semaphore
using System.Threading;

var meetingRooms = new Semaphore(2, 2);

void meeting(object coordinator)
{
    meetingRooms.WaitOne();

    Console.WriteLine($"{coordinator} is running the meeting");
    Thread.Sleep(3000);
    Console.WriteLine($"{coordinator}'s meeting is over");

    meetingRooms.Release();
}

var threads = new List<Thread>();
for (int i = 1; i <= 5; i++)
{
    Thread t = new Thread(meeting);
    t.Start(i);
    threads.Add(t);
}

foreach(var t in threads) t.Join();

Console.WriteLine("Day ended");

## RabbitMQ

- Exchange, Queue
- Pub/Sub, Consumer/Producer
- Enterprise Service Bus ?
- Kafka vs RabbitMQ
- RabbitMQ Performance / Througput Issues
    - Control Flow vs Data Flow

## What we achieved?

### Cons
- Dependencies increased
- Complexity increased
- Dev Testing became combursome
### Pros
- Modularity? How?
- Scalability? How?
    - https://developer.redis.com/operate/redis-at-scale
    - https://blog.rabbitmq.com/posts/2020/07/disaster-recovery-and-high-availability-101
- Portability
    - __Azure__; https://azure.microsoft.com/en-us/pricing/free-services
    - Web App for Containers; https://azure.microsoft.com/en-us/products/app-service/containers
    - Azure Container Instances; https://learn.microsoft.com/en-us/azure/container-instances
    - Azure Container Apps; https://learn.microsoft.com/en-us/azure/container-apps
    - Azure Kubernetes Service; https://learn.microsoft.com/en-us/azure/aks
    - https://learn.microsoft.com/en-us/azure/container-instances/tutorial-docker-compose